# Train model

### Imports

In [1]:
import os
if 'notebooks' in os.getcwd(): os.chdir('../..')  # change to main directory
print('Working directory:', os.getcwd() )

Working directory: /scratch/snx3000/bp000429/submission/adrian_sensorium


import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

from sensorium.utility.training import read_config, print_t, set_seed
from sensorium.utility import prediction

### Load configuration for model

In [3]:
config_file = 'notebooks/submission/config_submission_m2.yaml'
config = read_config( config_file )
print(config)

ordereddict([('data_sets', ['all']), ('dataset_fn', 'sensorium.datasets.static_loaders'), ('dataset_config', ordereddict([('normalize', True), ('include_behavior', True), ('include_eye_position', True), ('batch_size', 128), ('scale', 0.25), ('preload_from_merged_data', True), ('include_trial_id', True), ('include_rank_id', True), ('include_history', True), ('include_behav_state', True), ('adjusted_normalization', True)])), ('model_fn', 'sensorium.models.modulated_stacked_core_full_gauss_readout'), ('model_seed', 3452), ('model_config', ordereddict([('pad_input', False), ('stack', -1), ('layers', 4), ('input_kern', 9), ('gamma_input', 6.3831), ('gamma_readout', 0.0076), ('hidden_kern', 7), ('hidden_channels', 64), ('depth_separable', True), ('grid_mean_predictor', ordereddict([('type', 'cortex'), ('input_dimensions', 2), ('hidden_layers', 4), ('hidden_features', 20), ('nonlinearity', 'ReLU'), ('final_tanh', True)])), ('init_sigma', 0.1), ('init_mu_range', 0.3), ('gauss_type', 'full'), (

### Prepare dataloader

In [4]:
set_seed( config['model_seed'] )  # seed all random generators

if config['data_sets'][0] == 'all':
    basepath = "notebooks/data/"
    filenames = [os.path.join(basepath, file) for file in os.listdir(basepath) if ".zip" in file ]
    filenames = [file for file in filenames if 'static26872-17-20' not in file]
else:
    filenames = config['data_sets']
    # filenames like ['notebooks/data/static21067-10-18-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]
    
dataset_fn = config['dataset_fn']  # 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                  **config['dataset_config'],
                 }

dataloaders = get_data(dataset_fn, dataset_config)

### Instantiate model and trainer

In [5]:
# Instantiate model
model_fn = config['model_fn']     # e.g. 'sensorium.models.modulated_stacked_core_full_gauss_readout'
model_config = config['model_config']

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=config['model_seed'],
                 )

# Trainer
trainer_fn = config['trainer_fn']   # "sensorium.training.standard_trainer"
trainer_config = config['trainer_config']

trainer = get_trainer(trainer_fn=trainer_fn, 
                     trainer_config=trainer_config)

### Train model

In [6]:
print_t('Start of model training')
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)
print_t('Model training finished')

save_file = 'notebooks/submission/results/model_v2.pth'
torch.save(model.state_dict(), save_file )

2022-10-09 11:00:15.425493: Start of model training
correlation 0.0040906374
poisson_loss 28096714.0


Epoch 1: 100%|██████████| 216/216 [01:17<00:00,  2.79it/s]


correlation 0.16588514
poisson_loss 15558070.0


Epoch 2: 100%|██████████| 216/216 [01:17<00:00,  2.79it/s]


correlation 0.2173214
poisson_loss 14931198.0


...

correlation 0.46885255
poisson_loss 11808024.0


Epoch 113: 100%|██████████| 216/216 [01:17<00:00,  2.80it/s]


correlation 0.46894723
poisson_loss 11807556.0


Epoch 114: 100%|██████████| 216/216 [01:17<00:00,  2.80it/s]


2022-10-09 14:03:24.853475: Model training finished


### Save all predictions as .npy

In [7]:
if config['save_predictions_npy']:
    # calculate predictions per dataloader
    results = prediction.all_predictions_with_trial(model, dataloaders)

    # merge predictions, sort in time and add behavioral variables
    merged = prediction.merge_predictions(results)
    sorted_res = prediction.sort_predictions_by_time(merged)
    prediction.inplace_add_behavior_to_sorted_predictions(sorted_res)
    
    npy_file = 'notebooks/submission/results/prediction_model_v2.npy'
    np.save( npy_file, sorted_res)
    

Iterating datasets: 100%|██████████| 6/6 [01:17<00:00, 12.85s/it]
